In [26]:
!pip install vizdoom stable_baselines3 opencv-python gym

import random

import vizdoom
from stable_baselines3 import ppo
from stable_baselines3.common import policies



if __name__ == '__main__':
    
    # Instantiate a VizDoom game instance.
    game = vizdoom.DoomGame()
    game.load_config('basic.cfg')
    game.init()

    # Define possible actions. Each number represents the state of a button (1=active).
    sample_actions = [
        [1, 0, 0],  # Move left
        [0, 1, 0],  # Move right
        [0, 0, 1],  # Attack
    ]

    n_episodes = 10
    current_episode = 0
    
    while current_episode < n_episodes:
        game.make_action(random.choice(sample_actions))

        if game.is_episode_finished():
            current_episode += 1
            game.new_episode()

    game.close()

In [43]:
!pip install shimmy
import gym
from gym import spaces
import numpy as np
import vizdoom
from stable_baselines3.common.vec_env import DummyVecEnv, VecTransposeImage
from typing import Tuple, Dict

class DoomEnv(gym.Env):
    def __init__(self, scenario: str, frame_skip=4):
        super(DoomEnv, self).__init__()

        # Initialize VizDoom
        self.game = vizdoom.DoomGame()
        self.game.load_config(f"{scenario}.cfg")
        self.game.set_window_visible(False)
        self.game.set_mode(vizdoom.Mode.PLAYER)
        self.game.set_screen_format(vizdoom.ScreenFormat.RGB24)
        self.game.set_screen_resolution(vizdoom.ScreenResolution.RES_640X480)
        self.frame_skip = frame_skip
        self.game.init()

        # Define action and observation spaces
        self.action_space = spaces.Discrete(self.game.get_available_buttons_size())
        self.observation_space = spaces.Box(
            low=0,
            high=255,
            shape=(
                self.game.get_screen_height(),
                self.game.get_screen_width(),
                self.game.get_screen_channels(),
            ),
            dtype=np.uint8,
        )

    def reset(self):
        """Reset the environment to an initial state."""
        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        return state, {}  # Return observation and empty info dict

    def step(self, action: int) -> Tuple[np.ndarray, float, bool, Dict]:
        """Apply an action to the environment and return results."""
        reward = self.game.make_action(action, self.frame_skip)
        done = self.game.is_episode_finished()

        # Return a blank observation if the episode is done
        state = self.game.get_state().screen_buffer if not done else np.zeros(self.observation_space.shape, dtype=np.uint8)
        return state, reward, done, {}

    def render(self, mode="human"):
        """Optional render method."""
        pass

    def close(self):
        """Clean up resources."""
        self.game.close()

# Create a vectorized environment
def create_env(scenario: str, frame_skip:int) -> gym.Env:
    return DoomEnv(scenario=scenario, frame_skip=frame_skip)

def create_vec_env(**kwargs) -> VecTransposeImage:
    return VecTransposeImage(DummyVecEnv([lambda: create_env(**kwargs)]))

# Example usage
env_args = {
    'scenario': 'basic',
    'frame_skip': 4,
}

training_env = create_vec_env(**env_args)
obs = training_env.reset()
print("Initial Observation Shape:", obs.shape)

action = training_env.action_space.sample()
obs, reward, done, info = training_env.step(action)
print(f"Reward: {reward}, Done: {done}")


ValueError: The environment is of type <class '__main__.DoomEnv'>, not a Gymnasium environment. In this case, we expect OpenAI Gym to be installed and the environment to be an OpenAI Gym environment.

In [29]:
def create_env(scenario: str, **kwargs) -> DoomEnv:
    # Create a VizDoom instance.
    game = vizdoom.DoomGame()
    game.load_config(f'{scenario}.cfg')
    game.init()

    # Wrap the game with the Gym adapter.
    return DoomEnv(game, **kwargs)

from stable_baselines3.common.vec_env import DummyVecEnv, VecTransposeImage

def create_vec_env(**kwargs) -> VecTransposeImage:
    vec_env = DummyVecEnv([lambda: create_env(**kwargs)])
    return VecTransposeImage(vec_env)


In [40]:
!pip install shimmy
import cv2
from stable_baselines3 import ppo
from stable_baselines3.common import callbacks
from stable_baselines3.common import policies

import common

if __name__ == '__main__':
    # Create training and evaluation environments.
    env_args = {
        'scenario': 'basic',
        'frame_skip': 4,
        'frame_processor': lambda frame: cv2.resize(
            frame, None, fx=.5, fy=.5, interpolation=cv2.INTER_AREA)
    }

    # Create training and evaluation environments.
    training_env = create_vec_env(**env_args)
    eval_env = create_vec_env(**env_args)

    # Create an agent.
    agent = ppo.PPO(policy=policies.ActorCriticCnnPolicy,
                    env=training_env,
                    learning_rate=1e-4,
                    tensorboard_log='logs/tensorboard')

    # Add an evaluation callback that will save the best model when new records are reached.
    evaluation_callback = callbacks.EvalCallback(eval_env,
                                                 n_eval_episodes=10,
                                                 eval_freq=2500,
                                                 best_model_save_path='logs/models/basic')

    # Play!
    agent.learn(total_timesteps=25000, tb_log_name='ppo_basic', callback=evaluation_callback)

    training_env.close()
    eval_env.close()

TypeError: create_env() got an unexpected keyword argument 'frame_processor'

In [ ]:
def step(self, action: int) -> t.Tuple[Frame, int, bool, t.Dict]:
    """Apply an action to the environment and returns state, reward and done flag."""
    reward = self.game.make_action(self.possible_actions[action], self.frame_skip)
    done = self.game.is_episode_finished()
    self.state = self._get_frame(done)

    return self.state, reward, done, {}

def reset(self) -> Frame:
    """Resets the environment and returns the first observation."""
    
    self.game.new_episode()
    self.state = self._get_frame()

    return self.state

def _get_frame(self, done: bool = False) -> Frame:
    if not done:
        return self.frame_processor(self.game.get_state().screen_buffer)
    else:
        return self.empty_frame
    

NameError: name 't' is not defined